In [1]:
import os
import h5py
import numpy as np

In [3]:
def subsample_datasets(input_file, output_file, subsampling_factor):
    with h5py.File(input_file, 'r') as f_in:
        with h5py.File(output_file, 'w') as f_out:
            # Iterate over all datasets in the input file
            for dataset_name, dataset in f_in.items():
                shape = dataset.shape
                subsampled_shape = tuple(dim // subsampling_factor for dim in shape)
                subsampled_dataset = f_out.create_dataset(dataset_name, shape=subsampled_shape, dtype=dataset.dtype)

                # Calculate the number of elements to keep based on subsampling_factor
                num_elements = np.prod(subsampled_shape)
                num_elements_to_keep = num_elements * subsampling_factor

                # Reshape the original dataset to a 1D array
                original_data = np.reshape(dataset, (-1,))

                # Calculate the number of elements to truncate if necessary
                num_elements_remaining = len(original_data) % subsampling_factor
                if num_elements_remaining != 0:
                    original_data = original_data[:-num_elements_remaining]

                # Reshape the subsampled data to the subsampled shape
                subsampled_data = np.reshape(original_data, subsampled_shape)

                # Write the subsampled data to the subsampled dataset
                subsampled_dataset[...] = subsampled_data

                # Copy attributes from the original dataset to the subsampled dataset
                for attr_name, attr_value in dataset.attrs.items():
                    subsampled_dataset.attrs[attr_name] = attr_value

    print("Subsampling completed successfully.")

# Usage example
input_file = '/data/courseac/N-CMAPSS/data_set/N-CMAPSS_DS01-005.h5'
output_file = '/data/courseac/N-CMAPSS/subsampled/N-CMAPSS_DS01-005.h5'
subsampling_factor = 10  # Keep every 5th point

subsample_datasets(input_file, output_file, subsampling_factor)

ValueError: cannot reshape array of size 4906630 into shape (490663,1)